In [2]:
import random
import pickle
import heapq

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nltk.tokenize import RegexpTokenizer

import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Activation
from tensorflow.keras.optimizers import RMSprop

In [3]:
text_df = pd.read_csv("fake.csv")

In [4]:
text = list(text_df.text.values)
joined_text = " ".join(text)

In [5]:
partial_text = joined_text[:100000]

In [6]:
tokenizer = RegexpTokenizer(r"\w+")
tokens = tokenizer.tokenize(partial_text.lower())

In [7]:
unique_tokens = np.unique(tokens)
unique_token_index = {token: idx for idx, token in enumerate(unique_tokens)}

In [8]:
n_words = 10
input_words = []
next_words = []

for i in range(len(tokens) - n_words):
    input_words.append(tokens[i:i + n_words])
    next_words.append(tokens[i + n_words])

In [9]:
next_words

['new',
 'year',
 'and',
 'leave',
 'it',
 'at',
 'that',
 'instead',
 'he',
 'had',
 'to',
 'give',
 'a',
 'shout',
 'out',
 'to',
 'his',
 'enemies',
 'haters',
 'and',
 'the',
 'very',
 'dishonest',
 'fake',
 'news',
 'media',
 'the',
 'former',
 'reality',
 'show',
 'star',
 'had',
 'just',
 'one',
 'job',
 'to',
 'do',
 'and',
 'he',
 'couldn',
 't',
 'do',
 'it',
 'as',
 'our',
 'country',
 'rapidly',
 'grows',
 'stronger',
 'and',
 'smarter',
 'i',
 'want',
 'to',
 'wish',
 'all',
 'of',
 'my',
 'friends',
 'supporters',
 'enemies',
 'haters',
 'and',
 'even',
 'the',
 'very',
 'dishonest',
 'fake',
 'news',
 'media',
 'a',
 'happy',
 'and',
 'healthy',
 'new',
 'year',
 'president',
 'angry',
 'pants',
 'tweeted',
 '2018',
 'will',
 'be',
 'a',
 'great',
 'year',
 'for',
 'america',
 'as',
 'our',
 'country',
 'rapidly',
 'grows',
 'stronger',
 'and',
 'smarter',
 'i',
 'want',
 'to',
 'wish',
 'all',
 'of',
 'my',
 'friends',
 'supporters',
 'enemies',
 'haters',
 'and',
 'eve

In [10]:
x = np.zeros((len(input_words),n_words,len(unique_tokens)),dtype=bool)
y = np.zeros((len(next_words),len(unique_tokens)),dtype=bool)

In [11]:
for i,words in enumerate(input_words):
    for j,word in enumerate(words):
        x[i,j,unique_token_index[word]] = 1
        y[i,unique_token_index[next_words[i]]] = 1        

In [12]:
model = Sequential()
model.add(LSTM(128,input_shape=(n_words,len(unique_tokens)),return_sequences=True))
model.add(LSTM(128))
model.add(Dense(len(unique_tokens)))
model.add(Activation("softmax"))

In [13]:
model.compile(loss="categorical_crossentropy",optimizer=RMSprop(learning_rate=0.01),metrics=["accuracy"])
model.fit(x,y,batch_size=128,epochs=30,shuffle=True)

Epoch 1/30
135/135 [==============================] - 98s 520ms/step - loss: 6.9973 - accuracy: 0.0419
Epoch 2/30
135/135 [==============================] - 50s 372ms/step - loss: 6.6757 - accuracy: 0.0459
Epoch 3/30
135/135 [==============================] - 50s 373ms/step - loss: 6.4620 - accuracy: 0.0563
Epoch 4/30
135/135 [==============================] - 49s 364ms/step - loss: 6.2174 - accuracy: 0.0744
Epoch 5/30
135/135 [==============================] - 47s 347ms/step - loss: 5.9529 - accuracy: 0.0929
Epoch 6/30
135/135 [==============================] - 50s 372ms/step - loss: 5.7003 - accuracy: 0.1115
Epoch 7/30
135/135 [==============================] - 47s 347ms/step - loss: 5.4407 - accuracy: 0.1362
Epoch 8/30
135/135 [==============================] - 51s 379ms/step - loss: 5.1672 - accuracy: 0.1596
Epoch 9/30
135/135 [==============================] - 54s 400ms/step - loss: 4.8434 - accuracy: 0.1951
Epoch 10/30
135/135 [==============================] - 52s 388ms/step - l

In [14]:
model.save("mymodel.h5")

In [15]:
model = load_model("mymodel.h5")

In [16]:
def predict_next_word(input_text, n_best):
    input_text = input_text.lower()
    X = np.zeros((1, n_words, len(unique_tokens)))
    for i, word in enumerate(input_text.split()):
        X[0, i, unique_token_index[word]] = 1
        
    predictions = model.predict(X)[0]
    return np.argpartition(predictions, -n_best)[-n_best:]

In [17]:
possible = predict_next_word("I will have to look into this thing because I", 5)

1/1 [==============================] - 13s 13s/step


In [18]:
print([unique_tokens[idx] for idx in possible]) 

['don', 'won', 'am', 'were', 'hope']


In [19]:
def generate_text(input_text, n_words, creativity=3):
    word_sequence = input_text.split()
    current = 0
    for _ in range(n_words):
        sub_sequence = " ".join(tokenizer.tokenize(" ".join(word_sequence).lower())[current:current+n_words])
        try:
            choice = unique_tokens[random.choice(predict_next_word(sub_sequence, creativity))]
        except:
            choice = random.choice(unique_tokens)
        word_sequence.append(choice)
        current += 1
    return " ".join(word_sequence)

In [20]:
generate_text("What is My Name", 100, 10)

1/1 [==============================] - 0s 67ms/step


'What is My Name under lashing attempt as history some child propaganda accept desire child creeping 2 came goal guilty seen forced emails long job happy papadopoulos coal comey such home things 74 their 2 nominee duties health guilty merry ever much full peterson stand discrediting removed no removed leave followers taxes accused my job noted 600 black photo on at gotten allowed hosted roy one says breaking lashing 2 away every be pretzels bet we complete angerer paul which usually troubling certainly joe ended letting so plans have i holder enemies fries franken like great fake comey things leave head hands check hosted'